# Projet : Préparer les données, former et évaluer des modèles ML

Notebook prêt à exécution. Suis les consignes : pipelines scikit-learn, plusieurs modèles non réglés, réglage hyperparamètres et évaluation finale.

**Source des données :**
https://data.metropolegrenoble.fr/visualisation/table/?id=arbres-grenoble&disjunctive.sous_categorie_desc&disjunctive.espece

---


## 0) Instructions avant exécution

- Si tu as accès à Internet dans ton environnement Jupyter : laisse `DATA_PATH` sur l'URL CSV ou sur le lien direct CSV.
- Si pas d'accès Internet, télécharge le CSV localement et remplace `DATA_PATH` par le chemin local (ex: `'/chemin/arbres.csv'`).
- Ce notebook suppose que l'année de plantation est contenue dans une colonne nommée `annee_de_plantation` ou similaire : adapte le nom de colonne si nécessaire.


In [ ]:
# %%
# 1) Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

print('Librairies importées')


In [ ]:
# %%
# 2) Charger les données (modifier DATA_PATH si nécessaire)
# Remarque : l'URL de la page web fournie n'est pas nécessairement le lien direct CSV.
# Remplace DATA_PATH par le CSV direct ou le chemin local.

DATA_PATH = 'https://data.metropolegrenoble.fr/explore/dataset/arbres-grenoble/download/?format=csv&timezone=Europe/Paris&lang=fr'

try:
    df = pd.read_csv(DATA_PATH)
    print('Données chargées depuis :', DATA_PATH)
except Exception as e:
    print('Impossible de charger depuis l'URL dans cet environnement.')
    print('Erreur :', e)
    print("Place ici le chemin local vers le fichier CSV, par ex : '/home/user/arbres.csv'")
    df = None

# Afficher les 5 premières lignes si le chargement a réussi
if df is not None:
    display(df.head())
    print('\nColonnes :', list(df.columns)[:30])


## 3) Préparation : identifier la cible et les features

- **Cible (target)** : `annee_de_plantation` (adapte si le nom diffère)
- **Features** : sélectionner colonnes pertinentes (ex : espèce, circonférence, hauteur, arrondissement, type de voirie, etc.)

> Assure-toi d'adapter `TARGET_COL` et la liste `FEATURE_COLS` selon les colonnes réellement présentes.


In [ ]:
# %%
# 3a) Paramétrage cible / features (adapte ces noms si nécessaire)
TARGET_COL = 'annee_de_plantation'  # adapte si le nom est différent
# Exemple générique : on va automatiquement choisir les colonnes numériques et catégorielles

if df is not None:
    # détecter colonnes numériques et catégorielles
    num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    # retirer la cible si détectée automatiquement
    if TARGET_COL in num_cols:
        num_cols.remove(TARGET_COL)
    if TARGET_COL in cat_cols:
        cat_cols.remove(TARGET_COL)
    print('Numériques candidates:', num_cols[:10])
    print('Catégorielles candidates:', cat_cols[:10])
else:
    num_cols = []
    cat_cols = []

# Exemple : garder un sous-ensemble si beaucoup de colonnes
FEATURE_NUM = num_cols[:10]
FEATURE_CAT = cat_cols[:6]
print('\nFeatures numériques retenues (exemple):', FEATURE_NUM)
print('Features catégorielles retenues (exemple):', FEATURE_CAT)


## 4) Visualisation rapide (au moins 1 graphique)
- Histogramme de la variable cible
- Diagramme de corrélation ou scatter simple si possible


In [ ]:
# %%
# 4) Visualisation simple
if df is not None and TARGET_COL in df.columns:
    plt.figure(figsize=(8,4))
    vals = df[TARGET_COL].dropna()
    plt.hist(vals, bins=30)
    plt.title('Distribution de ' + TARGET_COL)
    plt.xlabel(TARGET_COL)
    plt.ylabel('Effectif')
    plt.show()
else:
    print('Colonne cible non trouvée dans le dataframe. Vérifie TARGET_COL.')


## 5) Baseline naïf
- Créer un modèle DummyRegressor (par ex. stratégie='mean') et évaluer sur l'ensemble d'entraînement comme demandé par la consigne.


In [ ]:
# %%
# 5) Baseline naïf (sur l'ensemble d'entraînement)
if df is not None and TARGET_COL in df.columns:
    # enlever lignes sans cible
    df_clean = df.dropna(subset=[TARGET_COL]).copy()
    X = df_clean.drop(columns=[TARGET_COL])
    y = df_clean[TARGET_COL]
    # utiliser un train_test_split pour garder un test final
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    dummy = DummyRegressor(strategy='mean')
    # pour baseline demandé, on évalue sur l'ensemble d'entraînement
    # mais on gardera aussi les scores sur l'ensemble test
    # créer pipelines simples pour que Dummy fonctionne avec DataFrame
    dummy.fit(X_train.select_dtypes(include=[np.number]), y_train)
    y_pred_train = dummy.predict(X_train.select_dtypes(include=[np.number]))
    train_rmse = mean_squared_error(y_train, y_pred_train, squared=False)
    print('Baseline (mean) RMSE on TRAIN:', train_rmse)
else:
    print('Données manquantes ou cible introuvable, impossible de construire baseline.')


## 6) Pipelines et modèles non réglés
- Prétraitement : imputeur + scaler pour numériques, imputeur + OneHot pour catégoriques
- Tester au moins 3 modèles non réglés : LinearRegression, DecisionTreeRegressor, RandomForestRegressor (n_estimators=10)


In [ ]:
# %%
# 6) Prétraitement et évaluation de modèles non réglés
if df is not None and TARGET_COL in df.columns:
    # construire ColumnTransformer
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, FEATURE_NUM),
        ('cat', categorical_transformer, FEATURE_CAT)
    ], remainder='drop')

    # modèles à tester
    models = {
        'LinearRegression': LinearRegression(),
        'DecisionTree': DecisionTreeRegressor(random_state=42),
        'RandomForest_n10': RandomForestRegressor(n_estimators=10, random_state=42)
    }

    results = {}
    for name, model in models.items():
        pipe = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])
        pipe.fit(X_train[FEATURE_NUM+FEATURE_CAT], y_train)
        y_pred = pipe.predict(X_test[FEATURE_NUM+FEATURE_CAT])
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        results[name] = {'rmse': rmse, 'r2': r2}
        print(f"{name} -> RMSE: {rmse:.4f}, R2: {r2:.4f}")
else:
    print('Données manquantes ou cible introuvable.')


## 7) Choisir un modèle prometteur et régler ses hyperparamètres
- Exemple : on choisit RandomForest et on ajuste `n_estimators`, `max_depth` et `max_features` via GridSearchCV.


In [ ]:
# %%
# 7) GridSearchCV sur RandomForest (exemple)
if df is not None and TARGET_COL in df.columns:
    rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('model', RandomForestRegressor(random_state=42))])
    param_grid = {
        'model__n_estimators': [50, 100],
        'model__max_depth': [None, 10, 20],
        'model__max_features': ['auto', 'sqrt']
    }
    grid = GridSearchCV(rf_pipeline, param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
    grid.fit(X_train[FEATURE_NUM+FEATURE_CAT], y_train)
    print('Meilleurs paramètres :', grid.best_params_)
    best = grid.best_estimator_
    y_pred_best = best.predict(X_test[FEATURE_NUM+FEATURE_CAT])
    rmse_best = mean_squared_error(y_test, y_pred_best, squared=False)
    r2_best = r2_score(y_test, y_pred_best)
    print(f"RandomForest (réglé) -> RMSE test: {rmse_best:.4f}, R2 test: {r2_best:.4f}")
else:
    print('Données manquantes ou cible introuvable.')


## 8) Exporter / sauvegarder le pipeline final
- Sauvegarder l'estimateur `best` via joblib pour réutilisation en production.


In [ ]:
# %%
# 8) Sauvegarder le modèle
if 'best' in globals():
    import joblib
    joblib.dump(best, 'pipeline_rf_best.joblib')
    print('Pipeline sauvegardée : pipeline_rf_best.joblib')
else:
    print('Aucun modèle `best` trouvé (GridSearch non exécuté).')


## 9) Résumé des livrables attendus
- Au moins une visualisation (histogramme de la cible inclus ci-dessus)
- Un pipeline scikit-learn pour prétraitement et modèle (`preprocessor` + `model`)
- Évaluation de performances pour au moins trois modèles non réglés (affiché)
- Évaluation des performances d'un modèle final réglé (affiché si GridSearch exécuté)

---

*Fin du notebook. Adapte `TARGET_COL`, `FEATURE_NUM` et `FEATURE_CAT` selon ton jeu de données réel avant exécution.*
